In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from fastai.collab import *
import pandas as pd

In [ ]:
products = pd.read_csv('/content/drive/My Drive/filtered_amazon_meta.csv')
products.columns.values

array(['Unnamed: 0', 'title', 'image', 'rank', 'main_cat', 'asin', 'description', 'also_view', 'also_buy', 'price',
       'similar_item', 'feature', 'tech1', 'date', 'brand'], dtype=object)

In [ ]:
products.shape

(947, 15)

In [ ]:
products.head()

,Unnamed: 0,title,image,rank,main_cat,asin,description,also_view,also_buy,price,similar_item,feature,tech1,date,brand
0,276,Shiseido THE SKINCARE Night Essential Moisturi...,['https://images-na.ssl-images-amazon.com/imag...,"3,208,315inBeautyamp;PersonalCare(",All Beauty,B000142SG4,"[""A multi-action emulsion Provides instant and...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,shiseido
1,276,Shiseido THE SKINCARE Night Essential Moisturi...,['https://images-na.ssl-images-amazon.com/imag...,"3,208,315inBeautyamp;PersonalCare(",All Beauty,B000142SG4,"[""A multi-action emulsion Provides instant and...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,shiseido
2,276,Shiseido THE SKINCARE Night Essential Moisturi...,['https://images-na.ssl-images-amazon.com/imag...,"3,208,315inBeautyamp;PersonalCare(",All Beauty,B000142SG4,"[""A multi-action emulsion Provides instant and...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,shiseido
3,276,Shiseido THE SKINCARE Night Essential Moisturi...,['https://images-na.ssl-images-amazon.com/imag...,"3,208,315inBeautyamp;PersonalCare(",All Beauty,B000142SG4,"[""A multi-action emulsion Provides instant and...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,shiseido
4,278,Yves Saint Laurent Instant Pur Natural Action ...,['https://images-na.ssl-images-amazon.com/imag...,"3,408,887inBeautyamp;PersonalCare(",All Beauty,B000142QMA,"[""A gentle exfoliating facial scrub Eases awa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,yves saint laurent


In [ ]:
data_selected = products[['asin','image','title','brand','description','rank','price']]
data_selected.sample(5)

,asin,image,title,brand,description,rank,price
352,B0051MQ3LW,NaN,"Philosophy Purity Cleansing Bar, 7 Ounce",philosophy,['Get back to the basics of cleansing. Purity ...,"1,047,871inBeautyPersonalCare(",NaN
560,B00ENL2O88,NaN,Cleansing Spa Water: Size - 480 ml,koh gen do,['As gentle as removing makeup with pure water...,"179,795inBeautyPersonalCare(",$62.00
874,B01A4I1TR0,NaN,"Petal Fresh, Ultra Moisturizing Facial Wipes, ...",petal fresh,['White Radiance Brightening Gently Soften & C...,"709,381inBeautyPersonalCare(",$8.25
387,B007N1AAJA,NaN,Omorovicza Rose Lifting Serum-1.01 oz,omorovicza,['Smooth and rejuvenate skin with the Omorovic...,"1,321,570inBeautyPersonalCare(",NaN
31,B000142QMA,['https://images-na.ssl-images-amazon.com/imag...,Yves Saint Laurent Instant Pur Natural Action ...,yves saint laurent,"[""A gentle exfoliating facial scrub Eases awa...","3,408,887inBeautyamp;PersonalCare(",NaN


**Data Preprocessing**

In [ ]:
data_selected_transformed = data_selected.dropna()

In [ ]:
data_selected_transformed.shape

(85, 7)

In [ ]:
data_selected_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85 entries, 268 to 941
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   asin         85 non-null     object
 1   image        85 non-null     object
 2   title        85 non-null     object
 3   brand        85 non-null     object
 4   description  85 non-null     object
 5   rank         85 non-null     object
 6   price        85 non-null     object
dtypes: object(7)
memory usage: 5.3+ KB


In [ ]:
data_selected_transformed.sample(2)

,asin,image,title,brand,description,rank,price
288,B002RZZXYE,['https://images-na.ssl-images-amazon.com/imag...,Kate Somerville ExfoliKate Intensive Exfoliati...,kate somerville,['Turn to Kate SomervilleExfoliKate CleanserDa...,"142,378inBeautyPersonalCare(",$175.00
295,B002RZZXYE,['https://images-na.ssl-images-amazon.com/imag...,Kate Somerville ExfoliKate Intensive Exfoliati...,kate somerville,['Turn to Kate SomervilleExfoliKate CleanserDa...,"142,378inBeautyPersonalCare(",$175.00


In [ ]:
#remove additional characters for rank
import re
def string_to_num(item):
  res = list(re.findall(r'\d+', item))
  num_str = ''
  for i in range(len(res)):
    num_str += res[i]
    rank_number = int(num_str)
  return rank_number

data_selected_transformed['rank'] = data_selected_transformed['rank'].apply(string_to_num)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
data_selected_transformed['rank']

268    219725
269    142378
270    142378
271    142378
272    142378
        ...  
936    766651
937    766651
938    766651
939    766651
941     26738
Name: rank, Length: 85, dtype: int64

In [ ]:
#replace $ for price 
data_selected_transformed['price'] = data_selected_transformed['price'].str.replace('$','')

# remove special chars for title
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–",'Â']

for char in spec_chars:
  data_selected_transformed['title'] = data_selected_transformed['title'].str.replace(char,'')

# remove [],''for image and description
spec_chars = ['[',']']

for char in spec_chars:
  data_selected_transformed['image'] = data_selected_transformed['image'].str.replace(char,'')
  data_selected_transformed['description'] = data_selected_transformed['description'].str.replace(char,'')

data_selected_transformed.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,asin,image,title,brand,description,rank,price
268,B002HG4JLQ,'https://images-na.ssl-images-amazon.com/image...,Peter Thomas Roth Instant Mineral SPF 45 012 O...,peter thomas roth,'A convenient brush on powder for instant powe...,219725,21.75
269,B002RZZXYE,'https://images-na.ssl-images-amazon.com/image...,Kate Somerville ExfoliKate Intensive Exfoliati...,kate somerville,'Turn to Kate SomervilleExfoliKate CleanserDai...,142378,175.00
270,B002RZZXYE,'https://images-na.ssl-images-amazon.com/image...,Kate Somerville ExfoliKate Intensive Exfoliati...,kate somerville,'Turn to Kate SomervilleExfoliKate CleanserDai...,142378,175.00
271,B002RZZXYE,'https://images-na.ssl-images-amazon.com/image...,Kate Somerville ExfoliKate Intensive Exfoliati...,kate somerville,'Turn to Kate SomervilleExfoliKate CleanserDai...,142378,175.00
272,B002RZZXYE,'https://images-na.ssl-images-amazon.com/image...,Kate Somerville ExfoliKate Intensive Exfoliati...,kate somerville,'Turn to Kate SomervilleExfoliKate CleanserDai...,142378,175.00


In [ ]:
data_selected_transformed['price'] = data_selected_transformed['price'].astype(float)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
data_selected_transformed.dtypes

asin            object
image           object
title           object
brand           object
description     object
rank             int64
price          float64
dtype: object

In [ ]:
item_dict ={}
df = data_selected_transformed[['asin','brand']].sort_values('asin').reset_index()

for i in range(df.shape[0]):
  item_dict[(df.loc[i,'asin'])] = df.loc[i,'brand']
  

In [ ]:
data_selected_transformed=data_selected_transformed[['asin','brand','rank','price']]

In [ ]:
# one hot encoding for categorial features 
data_selected_transformed = pd.get_dummies(data_selected_transformed,columns = ['brand'])
data_selected_transformed.head(5)

,asin,rank,price,brand_booty fresh,brand_kate somerville,brand_living proof,brand_moroccanoil,brand_murad,brand_ouai,brand_peter thomas roth,brand_philosophy,brand_rms beauty
268,B002HG4JLQ,219725,21.75,0,0,0,0,0,0,1,0,0
269,B002RZZXYE,142378,175.00,0,1,0,0,0,0,0,0,0
270,B002RZZXYE,142378,175.00,0,1,0,0,0,0,0,0,0
271,B002RZZXYE,142378,175.00,0,1,0,0,0,0,0,0,0
272,B002RZZXYE,142378,175.00,0,1,0,0,0,0,0,0,0


In [ ]:
data_selected_transformed = data_selected_transformed.sort_values('asin').reset_index().drop('index', axis=1)

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
# convert to csr matrix 
data_selected_transformed_csr = csr_matrix(data_selected_transformed.drop('asin', axis = 1).values)
data_selected_transformed_csr

<85x11 sparse matrix of type '<class 'numpy.float64'>'
	with 255 stored elements in Compressed Sparse Row format>

### **FastAI EmbeddingNN Model Approach**

In [ ]:
data = CollabDataBunch.from_df(data_selected_transformed,seed=42, valid_pct=0.1,user_name='asin', rating_name='price')

In [ ]:
data.show_batch()

asin,rank,target
B0085IFRYI,90721,11.79
B01GIQIBDC,766651,42.0
B002RZZXYE,142378,175.0
B002RZZXYE,142378,175.0
B002RZZXYE,142378,175.0


In [ ]:
data_selected_transformed.price.min(), data_selected_transformed.price.max()

(11.79, 175.0)

In [ ]:
learn = collab_learner(data, n_factors=40, y_range=(10, 175), wd=1e-1)

In [ ]:
learn = collab_learner(data, use_nn=True, emb_szs={'asin': 40, 'price': 40}, layers=[256, 128], y_range=(11.79,175.0))

In [ ]:
#find learning rate
learn.lr_find() 

epoch,train_loss,valid_loss,time
0,9043.518555,#na#,00:00
1,9021.446289,#na#,00:00
2,9015.036133,#na#,00:00
3,8983.330078,#na#,00:00
4,8943.783203,#na#,00:00
5,8918.318359,#na#,00:00
6,8900.365234,#na#,00:00
7,8876.813477,#na#,00:00
8,8881.390625,#na#,00:00
9,8927.038086,#na#,00:00


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [ ]:
learn.recorder.plot() # plot learning rate graph

In [ ]:
learn.fit_one_cycle(20, 1e-5)

epoch,train_loss,valid_loss,time
0,202.931580,839.127502,00:00
1,218.746765,822.786377,00:00
2,227.036499,834.998779,00:00
3,240.792770,801.301208,00:00
4,248.093689,795.653076,00:00
5,244.236252,785.965576,00:00
6,243.933334,787.930298,00:00
7,247.384903,835.863831,00:00
8,246.449341,836.402710,00:00
9,241.108246,872.933960,00:00


In [ ]:
learn.save('quinn-fastai')

### **LightFM Approach**

In [ ]:
!pip install lightfm

     |████████████████████████████████| 307kB 7.9MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=700105 sha256=dc294638e223c7c509b86f8c1e7916f77446b2d42f2aa1a9cfb6d06de14274ad
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [ ]:
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM

In [ ]:
split_data = random_train_test_split(data_selected_transformed_csr,test_percentage =0.2)
split_data

(<85x11 sparse matrix of type '<class 'numpy.float64'>'
 	with 204 stored elements in COOrdinate format>,
 <85x11 sparse matrix of type '<class 'numpy.float64'>'
 	with 51 stored elements in COOrdinate format>)

In [ ]:
model = LightFM(learning_rate=0.05,loss='warp',random_state=2018,
                no_components=150,
                user_alpha=0.000005)

model = model.fit(data_selected_transformed_csr, epochs=100, num_threads=16, verbose=False)

In [ ]:
auc= auc_score(model).mean()

TypeError: ignored